# Welcome to PYNQ

## Getting Started

To get started using PYNQ, try running the example notebooks in the folders described below. 

* **getting_started**: includes an introduction to using Jupyter notebook with PYNQ, the Python environment, and how to use some basic features of the current platform. 

* **common**: contains example notebooks on how to download an overlay, how to set the Zynq clocks, how to execute Linux shell commands, and how to use USB devices.

If other overlays or packages are installed, other folders with example notebooks may also be available in this directory.  


## Documentation

Please see the latest <a href="http://pynq.readthedocs.io">PYNQ Documentation on readthedocs</a>.  


## Support

For questions or support, go to the forum on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## Project webpage

You can find details on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## GitHub

The PYNQ Repository is hosted on github: <a href="https://github.com/Xilinx/PYNQ">PYNQ GitHub Repository</a>.

In [3]:
#Import PYNQ and its basic functions
import pynq
from pynq import Overlay
from pynq import allocate
import numpy as np
import os

In [4]:
#Os is really great is you don't know where to find bitstreams
os.listdir("/home/xilinx/jupyter_notebooks/")

['Welcome to Pynq Eclypse-Z7.ipynb',
 '.ipynb_checkpoints',
 'design_1.hwh',
 'design_1_adv.hwh',
 'design_1_adv.bit',
 'getting_started',
 'common',
 'design_1.bit']

In [5]:
#Create some overlay from design_1.bit and design_1.hwh ()
ol = Overlay("/home/xilinx/jupyter_notebooks/design_1.bit")
#List key IP'cores for example axi_dma, axi_gpio etc.
ol.ip_dict.keys()

dict_keys(['axi_dma_0', 'axi_gpio_0', 'axi_intc_0', 'mult_constant_0', 'add_0'])

In [6]:
#You can always assign those IP's to variables for easier acces
add = ol.add_0
mult_constant = ol.mult_constant_0
dma = ol.axi_dma_0

In [7]:
#Add IP will add two int numbers
print(add.register_map)
add.register_map.a = 3
add.register_map.b = 4
add.register_map.c

RegisterMap {
  a = Register(a=0),
  b = Register(b=0),
  c = Register(c=0),
  c_ctrl = Register(c_ap_vld=1, RESERVED=0)
}


Register(c=7)

In [8]:
#mult_constant IP will multiply in_buffer with constant_V variable
print(mult_constant.register_map)
in_buffer = allocate(shape=(5,), dtype=np.uint32)
out_buffer = allocate(shape=(5,), dtype=np.uint32)

for i in range(5):
    in_buffer[i] = i

mult_constant.register_map.constant_V = 3
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()

out_buffer

RegisterMap {
  constant_V = Register(constant_V=0)
}


PynqBuffer([ 0,  3,  6,  9, 12], dtype=uint32)

In [9]:
#Axi_gpio0 test, this doesn't work because of wrong pin's assignments in board
buttons = ol.axi_gpio_0.channel1
leds = ol.axi_gpio_0.channel2

In [10]:
leds.write(mask=0xFFFF, val=0x1)

In [11]:
#But as you can see IP core is working 
leds.read()

1

In [12]:
################################################################################

In [13]:
#Let's load another bitstream, This bitstream has addfloat IP core
#it works just like add in previous example but this time on floats
ol_adv = Overlay("/home/xilinx/jupyter_notebooks/design_1_adv.bit")
ol_adv.ip_dict.keys()

dict_keys(['axi_dma_mult_constant', 'axi_gpio_0', 'axi_intc_0', 'mult_constant_0', 'add_0', 'addfloat_0'])

In [14]:
#This will turn off board leds
mask = 0xffffff
ol_adv.axi_gpio_0.channel2.write(0x0, mask)
addfloat = ol_adv.addfloat_0

In [15]:
#Let's create translate functions that will turn float values into uint32
import struct
def float_to_uint(f):
    return int(struct.unpack('<I', struct.pack('<f', f))[0])
def uint_to_float(f):
    return float(struct.unpack('<f', struct.pack('<I', f))[0])

In [16]:
#Assign values
a = np.float32(-1.5)
b = np.float32(1.25)
aint = float_to_uint(a)
bint = float_to_uint(b)

In [17]:
addfloat.register_map.a = aint 
addfloat.register_map.b = bint
cint = addfloat.register_map.c

print(uint_to_float(cint))

-0.25


In [ ]:
#All IP's sources are located in github https://github.com/bartokon/Eclypse-Z7-Notebooks